# [Module 3.2] Personalize 솔류션 버전 평가 지표 확인하기

이 노트북은 아래와 같은 작업을 합니다.

* 솔류션 버전으로 부터 성능 지표 구하기
* 성능 지표 분석

---
노트북의 실행 소요 시간은 약 1분 입니다.

In [1]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time


다음으로 여러분의 환경이 Amazon Personalize와 성공적으로 통신할 수 있는지 확인해야 합니다.

In [2]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')

아래 코드 셀은 이전 notebook에서 저장했던 공유 변수들을 불러옵니다.

In [3]:
%store -r

생성할 오브젝트의 끝에 임의의 숫자를 부여하기 위해 suffix 정의

## 솔루션 버전 평가를 위한 학습 데이터 분리 방법

이번 파트에서는 Amazon Personalize에서 기본으로 제공하는 솔루션에 대한 평가 지표를 확인해 봅니다. 
Amazon Personalize에서는 평가 지표를 생성하기 위해 약 랜덤으로 10% 사용자의 interaction data를 테스트 용으로 활용합니다. 

아래 이미지는 Amazon Personalize가 데이터를 분리하는 방법을 보여줍니다. 사용자가 10 명이고 각각 10 개의 상호 작용이있는 경우 (여기에서 원은 Interaction data를 나타냄) 타임 스탬프를 기준으로 가장 오래된 것부터 최신 것까지 나열된 것입니다. Amazon Personalize는 사용자의 90 % (파란색 원)의 모든 Interaction 데이터를 사용하여 솔루션 버전을 훈련시키고 나머지 10 %는 평가를 위해 사용합니다. 나머지 10 %의 각 사용자에 대해 Interaction data (녹색 원)의 90 %가 훈련 된 모델의 입력값으로 사용됩니다. 데이터의 나머지 10 % (주황색 원)는 모델에서 생성 된 추천 결과물과 비교되고 평가 지표를 계산하는 데 사용됩니다.



![personalize metrics](static/imgs/personalize_metrics.png)



#### 솔류션 버전의 성능 지표 구하기



In [4]:
metrics=[]

def build_metric_matrix(solution,response):
    metrics.append([solution,
                response['metrics']['coverage'],
                response['metrics']['mean_reciprocal_rank_at_25'],
                response['metrics']['normalized_discounted_cumulative_gain_at_5'],
                response['metrics']['normalized_discounted_cumulative_gain_at_10'],
                response['metrics']['normalized_discounted_cumulative_gain_at_25'],
                response['metrics']['precision_at_5'],
                response['metrics']['precision_at_10'],
                response['metrics']['precision_at_25']])

#### Metrics: Popularity

아래는 생성한 7개의 솔류션 버전에 대해서 성능 지표를 얻습니다.

In [5]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = popularity_solution_version_arn 
)

print(json.dumps(get_solution_metrics_response, indent=2))
build_metric_matrix('popularity',get_solution_metrics_response)


{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-popularity-60498/a7709e59",
  "metrics": {
    "coverage": 0.0136,
    "mean_reciprocal_rank_at_25": 0.0456,
    "normalized_discounted_cumulative_gain_at_10": 0.0606,
    "normalized_discounted_cumulative_gain_at_25": 0.0981,
    "normalized_discounted_cumulative_gain_at_5": 0.0362,
    "precision_at_10": 0.0163,
    "precision_at_25": 0.015,
    "precision_at_5": 0.0143
  },
  "ResponseMetadata": {
    "RequestId": "7457ef34-282e-4117-aa7c-9902109ff641",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 07 Sep 2020 10:15:33 GMT",
      "x-amzn-requestid": "7457ef34-282e-4117-aa7c-9902109ff641",
      "content-length": "414",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Metrics: User-Personalization

In [6]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = user_personalization_solution_version_arn
)

print(json.dumps(get_solution_metrics_response, indent=2))

build_metric_matrix('user_personalization',get_solution_metrics_response)

{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-user-personalization-60498/2ea58de0",
  "metrics": {
    "coverage": 0.2959,
    "mean_reciprocal_rank_at_25": 0.2335,
    "normalized_discounted_cumulative_gain_at_10": 0.2301,
    "normalized_discounted_cumulative_gain_at_25": 0.3161,
    "normalized_discounted_cumulative_gain_at_5": 0.1751,
    "precision_at_10": 0.082,
    "precision_at_25": 0.0616,
    "precision_at_5": 0.0942
  },
  "ResponseMetadata": {
    "RequestId": "1c587611-8b78-4196-b772-be29c15a4254",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 07 Sep 2020 10:15:33 GMT",
      "x-amzn-requestid": "1c587611-8b78-4196-b772-be29c15a4254",
      "content-length": "424",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Metrics: HRNN

In [7]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = hrnn_solution_version_arn
)

print(json.dumps(get_solution_metrics_response, indent=2))

build_metric_matrix('hrnn',get_solution_metrics_response)

{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-aws-hrnn-60498/b1d26512",
  "metrics": {
    "coverage": 0.6349,
    "mean_reciprocal_rank_at_25": 0.2438,
    "normalized_discounted_cumulative_gain_at_10": 0.2456,
    "normalized_discounted_cumulative_gain_at_25": 0.3312,
    "normalized_discounted_cumulative_gain_at_5": 0.1991,
    "precision_at_10": 0.0824,
    "precision_at_25": 0.0627,
    "precision_at_5": 0.1013
  },
  "ResponseMetadata": {
    "RequestId": "68804574-2e87-4079-a68a-3a258c20a1a9",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 07 Sep 2020 10:15:33 GMT",
      "x-amzn-requestid": "68804574-2e87-4079-a68a-3a258c20a1a9",
      "content-length": "413",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Metrics: HRNN-Meta

In [8]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = hrnn_meta_solution_version_arn
)

print(json.dumps(get_solution_metrics_response, indent=2))
build_metric_matrix('hrnn_meta',get_solution_metrics_response)

{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-aws-hrnn-metadata-60498/765fd88d",
  "metrics": {
    "coverage": 0.5898,
    "mean_reciprocal_rank_at_25": 0.2427,
    "normalized_discounted_cumulative_gain_at_10": 0.2354,
    "normalized_discounted_cumulative_gain_at_25": 0.3281,
    "normalized_discounted_cumulative_gain_at_5": 0.1884,
    "precision_at_10": 0.077,
    "precision_at_25": 0.0613,
    "precision_at_5": 0.0961
  },
  "ResponseMetadata": {
    "RequestId": "e4a79fc0-c866-4053-892c-67acd0addd7d",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 07 Sep 2020 10:15:34 GMT",
      "x-amzn-requestid": "e4a79fc0-c866-4053-892c-67acd0addd7d",
      "content-length": "421",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Metrics: HRNN-Coldstart

In [9]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = hrnn_coldstart_solution_version_arn
)

print(json.dumps(get_solution_metrics_response, indent=2))
build_metric_matrix('hrnn_coldstart',get_solution_metrics_response)

{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-hrnn-coldstart-60498/7e3c69db",
  "metrics": {
    "coverage": 0.229,
    "mean_reciprocal_rank_at_25": 0.0038,
    "normalized_discounted_cumulative_gain_at_10": 0.0044,
    "normalized_discounted_cumulative_gain_at_25": 0.0077,
    "normalized_discounted_cumulative_gain_at_5": 0.0032,
    "precision_at_10": 0.0006,
    "precision_at_25": 0.0008,
    "precision_at_5": 0.0006
  },
  "ResponseMetadata": {
    "RequestId": "c14a6af7-795a-44c1-a242-746f5021903f",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 07 Sep 2020 10:15:33 GMT",
      "x-amzn-requestid": "c14a6af7-795a-44c1-a242-746f5021903f",
      "content-length": "418",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Metrics: SIMS

In [10]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = sims_solution_version_arn
)

print(json.dumps(get_solution_metrics_response, indent=2))
build_metric_matrix('sims',get_solution_metrics_response)

{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-sims-60498/3d08ae29",
  "metrics": {
    "coverage": 0.7622,
    "mean_reciprocal_rank_at_25": 0.1807,
    "normalized_discounted_cumulative_gain_at_10": 0.1834,
    "normalized_discounted_cumulative_gain_at_25": 0.2516,
    "normalized_discounted_cumulative_gain_at_5": 0.151,
    "precision_at_10": 0.0533,
    "precision_at_25": 0.0404,
    "precision_at_5": 0.0686
  },
  "ResponseMetadata": {
    "RequestId": "56e103e0-958f-403f-b5c1-04098061a37f",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 07 Sep 2020 10:15:33 GMT",
      "x-amzn-requestid": "56e103e0-958f-403f-b5c1-04098061a37f",
      "content-length": "408",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Metrics: Ranking

In [11]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = ranking_solution_version_arn 
)

print(json.dumps(get_solution_metrics_response, indent=2))
build_metric_matrix('ranking',get_solution_metrics_response)

{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-ranking-60498/f669db6d",
  "metrics": {
    "coverage": 0.0136,
    "mean_reciprocal_rank_at_25": 0.0865,
    "normalized_discounted_cumulative_gain_at_10": 0.1149,
    "normalized_discounted_cumulative_gain_at_25": 0.1379,
    "normalized_discounted_cumulative_gain_at_5": 0.0928,
    "precision_at_10": 0.0236,
    "precision_at_25": 0.0145,
    "precision_at_5": 0.0299
  },
  "ResponseMetadata": {
    "RequestId": "86003d4b-3261-4921-8651-fe99e69ffc89",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 07 Sep 2020 10:15:33 GMT",
      "x-amzn-requestid": "86003d4b-3261-4921-8651-fe99e69ffc89",
      "content-length": "412",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


## Summary Metrics

#### 1. 솔류션 평가 지표 정의
[솔류션 평가 지표 정의](https://docs.aws.amazon.com/personalize/latest/dg/working-with-training-metrics.html)
는 개발자 문서의 링크 참조 바랍니다. 또한 이 링크 [솔류션 평가 정의 예제](http://francescopochetti.com/recommend-expedia-hotels-with-amazon-personalize-the-magic-of-hierarchical-rnns/) 의 페이지 맨 아래 쪽을 보시면 조금 더 직관적인 그림을 보실 수 있습니다.
 <br>
또한 reciprocal_rank_at_5, normalized_discounted_cumulative_gain_at_5,precision_at_5 의 예제는 아래와 같습니다. 
* Exmaple
    * 5 개의 추천리스트를 제공했고, 이 중에 2번째와 5번째가 실제 데이타와 일치 했다고 하면, 쉽게 이렇게 [0,1,0,0,1] 표시 할 수 있습니다.
        * reciprocal_rank
            * 1/2 (0.5) # 가장 빠른 순서의 하나만을 선택 합니다
        * normalized_discounted_cumulative_gain_at_5
            * (1/log(1+2) + 1/log(1+5)) / (1/log(1+1) + 1/log(1+2)) = 0.6241
        * precision_at_5
            * 2/5 (0.4)

#### 2. 솔류션 버전의 성능 지표 분석

레서피의 종류는 크게 세가지 입니다. 각각에 대해 확인을 해보겠습니다.
#### 1. USER_PERSONALIZATION Recipes
- 5가지 (popularity,user_personalization,hrnn,hrnn_meta,hrnn_coldstart) 있습니다.
- popularity 는 베이스라인의 레서피로서 샤용을 주로 합니다. 모든 지표에서 가장 낮은 수치를 보입니다.
- hrnn, hrnn-meta, user_personalization 이 전반적으로 가장 높은 성능을 보여 줍니다. user_personalization은 default로 exploration_weight=0.3 입니다. coldstart item 이 거의 없는 상태에서도 높은 성능을 보여 주고 있습니다.학습 데이터의 인터렉션 데이터에 콜드 스타트 아이템이 전형 없기에 성능은 감소합니다. 이 경우에는 exloration_weight = 0을 하게 되면 콜드 스타트의 아이템을 배제하고 추천 합니다.<br>
이 링크 [exploration_weight](https://docs.aws.amazon.com/personalize/latest/dg/native-recipe-new-item-USER_PERSONALIZATION.html)에서 "Item Exploration Configuration Hyperparameters" 이 부분을 참고 하세요.<br>
- hrnn_coldstart 는coldstart item 이 거의 없는 상태이기에 성능이 낮게 나오는 것이 정상 으로 보입니다.

#### 2. RELATED_ITEMS Recipes
- sims 가 부류에 속하는 레서피로 커버리지가 높게 나왔습니다.

#### 3. PERSONALIZED_RANKING Recipes
- ranking 이 여기에 속합니다. 


<div>
    <img src="static/imgs/Fig.3.2.metric_summary2.png" width="800">
</div>

---
참고:
**<font color="orange">* 추천시에 시간에 대한 입력값이 들어가기에 추천 결과가 아래 이미지 (저자 실행)와 다를 수 있습니다. 여러 번 테스트 시에 매번 조금씩 다르게 나옴을 확인 하였습니다.</font>** <br>
레서비의 정의는 개발자 문서 참고 하세요 --> [레서피 정의](https://docs.aws.amazon.com/personalize/latest/dg/working-with-predefined-recipes.html)


In [12]:
recipe_metrics=pd.DataFrame(metrics,columns=['recipe','coverage','mrr@25','ndcg@5','ndcg@10','ndcg@25','p@5','p@10','p@25'])

recipe_metrics

,recipe,coverage,mrr@25,ndcg@5,ndcg@10,ndcg@25,p@5,p@10,p@25
0,popularity,0.0136,0.0456,0.0362,0.0606,0.0981,0.0143,0.0163,0.0150
1,user_personalization,0.2959,0.2335,0.1751,0.2301,0.3161,0.0942,0.0820,0.0616
2,hrnn,0.6349,0.2438,0.1991,0.2456,0.3312,0.1013,0.0824,0.0627
3,hrnn_meta,0.5898,0.2427,0.1884,0.2354,0.3281,0.0961,0.0770,0.0613
4,hrnn_coldstart,0.2290,0.0038,0.0032,0.0044,0.0077,0.0006,0.0006,0.0008
5,sims,0.7622,0.1807,0.1510,0.1834,0.2516,0.0686,0.0533,0.0404
6,ranking,0.0136,0.0865,0.0928,0.1149,0.1379,0.0299,0.0236,0.0145


In [13]:
%store user_personalization_solution_version_arn
%store user_personalization_solution_arn


%store hrnn_solution_version_arn
%store hrnn_solution_arn

%store hrnn_meta_solution_version_arn
%store hrnn_meta_solution_arn

%store hrnn_coldstart_solution_version_arn
%store hrnn_coldstart_solution_arn

%store sims_solution_version_arn
%store sims_solution_arn


Stored 'user_personalization_solution_version_arn' (str)
Stored 'user_personalization_solution_arn' (str)
Stored 'hrnn_solution_version_arn' (str)
Stored 'hrnn_solution_arn' (str)
Stored 'hrnn_meta_solution_version_arn' (str)
Stored 'hrnn_meta_solution_arn' (str)
Stored 'hrnn_coldstart_solution_version_arn' (str)
Stored 'hrnn_coldstart_solution_arn' (str)
Stored 'sims_solution_version_arn' (str)
Stored 'sims_solution_arn' (str)
